# Déploiement ACI (Azure Container Instance)

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

> Documentation: https://docs.microsoft.com/en-us/azure/machine-learning/

## 1. Informations

In [ ]:
import sys
print("Python version : ",sys.version)

In [ ]:
import datetime
maintenant = datetime.datetime.now()
print(maintenant)

In [ ]:
#Version
import azureml.core
print("Version Azure ML service :", azureml.core.VERSION)

In [ ]:
import azureml.core
from azureml.core.workspace import Workspace
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from azureml.core.model import Model
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from azureml.core import Environment
from azureml.core.model import InferenceConfig

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep='\n')

## 3. Prédiction des prix des biens immobiliers

### Boston dataset example

Feature Descriptions				
1. CRIM - per capita crime rate by town				
2. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.				
3. INDUS - proportion of non-retail business acres per town.				
4. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)				
5. NOX - nitric oxides concentration (parts per 10 million)				
6. RM - average number of rooms per dwelling				
7. AGE - proportion of owner-occupied units built prior to 1940				
8. DIS - weighted distances to five Boston employment centres				
9. RAD - index of accessibility to radial highways				
10. TAX - full-value property-tax rate per $10000	

11. PTRATIO - pupil-teacher ratio by town				
12. B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town				
13. LSTAT - pct lower status of the population				
14. MEDV - Median value of owner-occupied homes in $1000's

In [ ]:
boston = load_boston()
bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
bos['PRICE'] = boston.target

In [ ]:
bos.head()

In [ ]:
bos.describe()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
boston = load_boston()
plt.figure(figsize=(5, 4))
plt.hist(boston.target)
plt.title('Graphique')
plt.xlabel('price ($1000s)')
plt.ylabel('count')
plt.show()

In [ ]:
import seaborn as sns
 
boston = load_boston()
boston_pd = pd.DataFrame(boston.data, columns = boston.feature_names)
correlation_matrix = boston_pd.corr().round(2)
sns.heatmap(correlation_matrix, cmap="YlGnBu")
plt.show()

In [ ]:
for index, feature_name in enumerate(boston.feature_names):
    plt.figure(figsize=(5, 4))
    plt.scatter(boston.data[:, index], boston.target)
    plt.ylabel('Price', size=12)
    plt.xlabel(feature_name, size=12)
    plt.show()

In [ ]:
X = bos.drop('PRICE', axis = 1)
Y = bos['PRICE']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 5)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, Y_train)

In [ ]:
print('Coefficients: \n', lm.coef_)

In [ ]:
import numpy as np
print('R2 Test =', np.round(lm.score(X_test,Y_test)*100,2))
print('R2 Training =', np.round(lm.score(X_train,Y_train)*100,2))

In [ ]:
Y_pred = lm.predict(X_test)
plt.scatter(Y_test, Y_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()

### Référencement du modèle de ML

In [ ]:
model_file_name = 'boston_model.pkl'
joblib.dump(value = lm, filename = model_file_name)

registered_trained_model = Model.register(workspace=ws, 
                                          model_path=model_file_name, 
                                          model_name=model_file_name, 
                                          description="Modèle Pricing Boston", 
                                          tags = {'algo': "Regression", 'Training context': "Azure ML", 'Framework' : 'scikit-learn'}
                                         )

In [ ]:
print('Nom du modèle :', registered_trained_model.name)

In [ ]:
print('Version du modèle :', registered_trained_model.version)

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, '- version =', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 4. Déploiement Azure Container Instance (ACI)

### 4.1 Création du scoring file

In [ ]:
%%writefile scoreboston.py
import pickle
import json
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # note here "sklearn_regression_model.pkl" is the name of the model registered under
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('boston_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

input_sample = np.array([[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()[0]
    except Exception as e:
        error = str(e)
        return error

In [ ]:
%ls scoreboston.py -l

### 4.2 Fichier environnement

In [ ]:
%%writefile myenvboston.yml
name: project_environment
dependencies:
  - python=3.6.9
  - pip:
    - azureml-defaults
    - scikit-learn
    - numpy
    - inference-schema[numpy-support]

In [ ]:
#Viewing the yml file
with open(os.path.join('./myenvboston.yml'), 'r') as f:
    print(f.read())

In [ ]:
%ls myenvboston.yml -l

In [ ]:
env = Environment.from_conda_specification(name='deploytocloudenv', file_path='myenvboston.yml')
inference_config = InferenceConfig(entry_script="scoreboston.py", environment=env)

### 4.3 Déploiement du modèle
> Prévoir 3 minutes de temps de traitement

Documentation : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-deploy-azure-container-instance

In [ ]:
%%time
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                       memory_gb=1,
                                                       auth_enabled=False,
                                                       ssl_enabled=False,
                                                       enable_app_insights=True)

aci_service_name = 'bostonprice-aci'

try:
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [registered_trained_model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

> Le endpoint est visible depuis **Azure ML Studio**. <br>
Nous pouvons également visualiser le container créé dans les **resources ACI** du portail Azure.

In [ ]:
print("Informations du modèle déployé en ACI :")
print(" - Endpoint :", service.scoring_uri)
print(" - Statut :", service.state)
print(" - Swagger :", service.swagger_uri)

In [ ]:
print("Modèles déployés avec Azure ML :")
for webservice_name in ws.webservices:
    print(webservice_name)

In [ ]:
#print("Logs :")
#print(service.get_logs())

## 5. Test du modèle déployé

> Documentation : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-consume-web-service

In [ ]:
print("Voici le scoring endpoint: ", service.scoring_uri)

### 5.1 Premier test

In [ ]:
import json
test1 = json.dumps({'data': [
    [0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98]]})

In [ ]:
test1

In [ ]:
%%time
test1encode = bytes(test1, encoding='utf8')

print("Données en entrée : ")
print(test1encode)
prediction1 = service.run(test1encode)
print()
print("=> Prédiction = ", round(prediction1, 5))

### 5.2 Second test

In [ ]:
import json
test2 = json.dumps({'data': [
    [0.00432,18.0,1.1,0.12,0.422,6.75,65.2,4.0900,1.0,296.0,16.3,496.90,6.8]]})

In [ ]:
test2

In [ ]:
%%time
test2encode = bytes(test2, encoding='utf8')

print("Données en entrée : ")
print(test2encode)
prediction2 = service.run(input_data=test2encode)
print()
print("=> Prédiction = ", round(prediction2, 5))

## 6. Test avec Postman

https://www.postman.com

### Paramétrage Postman
1. Post : endpoint
2. Content-Type : application/json
3. A spécifier dans le Body :

<img src="https://github.com/retkowsky/images/blob/master/postman1.jpg?raw=true">

## 7. Télémétrie
> Aller ensuite dans **AppInsights** dans le ressource group Azure ML. Bouton **Logs** pour voir les traces.

<img src="https://github.com/retkowsky/images/blob/master/appinsightsaml.jpg?raw=true">

In [ ]:
# Pour supprimer le service
#service.delete()

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">